# Citywide Crash Predictive ML Model

We will create a Logistic Regression or Tree based model to predict the daily number of crashes at a known intersection using the following:
- weather (temp, snow, rain, precip)
- day of week (Mon, Tues...)
- intersection characteristics
    - speed limit
    - volume of traffic
    - region
    - lanes/ways/turns/split/exit etc.
- violations (may want a violations independent model)
- red light cam present

In [1]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from modules.myfuncs import *
from sklearn.model_selection import GridSearchCV, cross_val_score


import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')


# Decision Tree
import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support


import pandas as pd
import numpy as np
import itertools

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE, ADASYN

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [2]:
conn = create_connection('database/rlc.db')  # function from myfuncs file
c = conn.cursor()

sqlite3 version: 2.6.0
connected to database/rlc.db



We will try to get data that is daily for all of the above info.  

In [3]:
print(sql_fetch_tables(c, conn))



def table_info(c, conn):
    '''
    prints out all of the columns of every table in db
    c : cursor object
    conn : database connection object
    '''
    tables = c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    for table_name in tables:
        table_name = table_name[0]
        table = pd.read_sql_query("SELECT * from {} LIMIT 0".format(table_name), conn)
        print(table_name)
        for col in table.columns:
            print('\t' + col)
        print()

table_info(c, conn)


[('cam_locations',), ('cam_startend',), ('daily_violations',), ('all_crashes',), ('hourly_congestion',), ('hourly_weather',), ('region_data',), ('intersection_cams',), ('all_hours',), ('int_startend',), ('intersection_chars',), ('signal_crashes',), ('rlc_all_hours',), ('daily_covid',), ('holidays',)]
cam_locations
	camera_id
	intersection
	address
	violation_date
	violations
	latitude
	longitude
	month
	day
	weekday
	year

cam_startend
	camera_id
	start
	end

daily_violations
	intersection
	camera_id
	address
	violation_date
	violations
	latitude
	longitude
	month
	day
	weekday
	year

all_crashes
	crash_record_id
	rd_no
	crash_date
	posted_speed_limit
	traffic_control_device
	device_condition
	weather_condition
	lighting_condition
	first_crash_type
	trafficway_type
	alignment
	roadway_surface_cond
	road_defect
	report_type
	crash_type
	damage
	prim_contributory_cause
	sec_contributory_cause
	street_no
	street_direction
	street_name
	beat_of_occurrence
	num_units
	most_severe_injury
	in

## Query our data
I would like to get hourly intersection data with the following columns:


signal_crashes
	-crash_date
	-posted_speed_limit
	-device_condition
	-weather_condition
	-lighting_condition
	-first_crash_type
	-trafficway_type
	-alignment
	-roadway_surface_cond
	-road_defect
	-report_type
	-crash_type
	-damage
	-prim_contributory_cause
	-sec_contributory_cause
	-street_no
	-street_direction
	-street_name
	-most_severe_injury
	-injuries_total
	-injuries_fatal
	-injuries_incapacitating
	-injuries_non_incapacitating
	-injuries_reported_not_evident
	-injuries_no_indication
	-injuries_unknown
	-latitude
	-longitude
	-lane_cnt
	-work_zone_i
	-work_zone_type
	-workers_present_i
	-intersection
	-year
	-month
	-day
	-hour
	-region_id
	-time
	-weekday

hourly_weather
	-temp
	-rain_1h
	-rain_3h
	-snow_1h
	-snow_3h
	-time
	-year
	-month
	-day
	-hour
	-weekday

hourly_congestion
	-year
	-month
	-day
	-hour
	-region_id
	-bus_count
	-num_reads
	-speed
	-weekday
    
Tables will be queried to JOIN on year, month, day, hour

WHAT I REALLY NEED HERE
A single dataset with the following:
- daily intersection chars (all the stuff I entered)
- daily intersection crashes (injuries, deaths, number etc)
- daily wx (temp, precip mainly)
- daily congestion (might not need this)
- daily violations (already in the format)

In [4]:
# Approximately how many entries should I expect to be looking at after filtering my data
# 3 years, 4 months x 153 intersections
print((365.25*3 + 30*4) *153)


186009.75


## Query my db to get a DataFrame with crashes and intersection
We will be using the number of crashes and injuries to do some t-tests

In [ ]:
# To change from citywide to signal crashes only, change cr query from all_crashes to signal_crashes (line 31)
# Query takes 5+ minutes

big_df = pd.read_sql_query('''
WITH v AS(SELECT  SUM(violations) as violations,
            year,
            month,
            day

    FROM daily_violations
    GROUP BY year, month, day
    ),

    ah AS(SELECT year,
                month,
                day,
                hour,
                datetime
            FROM all_hours
            GROUP BY year, month, day, hour
        ), 

    cr AS(SELECT strftime('%m',crash_date) AS month,
                strftime('%Y',crash_date) AS year,
                strftime('%d',crash_date) AS day,
                strftime('%H',crash_date) AS hour,
                crash_record_id,
                injuries_total,
                injuries_fatal
                
         FROM all_crashes
        ),

    wx AS(SELECT MAX(temp) AS temp,
                (SUM(snow_1h) + SUM(rain_1h)) AS precip,
                SUM(snow_1h) AS snow_1h,
                SUM(rain_1h) AS rain_1h,
                year,
                month,
                day,
                hour
        FROM hourly_weather
        GROUP BY year, month, day, hour
        ),
    
    cov AS(SELECT strftime('%m',lab_report_date) AS month,
                strftime('%Y',lab_report_date) AS year,
                strftime('%d',lab_report_date) AS day,
                lab_report_date,
                cases_total
            FROM daily_covid as dc
            GROUP BY year, month, day
            ),
            
     cg AS(SELECT 
             (100 - AVG(quantile_speed))/100 as congestion,
             year,
             month,
             day,
             hour
         FROM hourly_congestion
         GROUP BY year, month, day, hour
     )
            
    SELECT  strftime('%W',ah.datetime) AS week,
            strftime('%w',ah.datetime) AS weekday,
            ah.datetime,
            ah.year,
            ah.month,
            ah.day,
            ah.hour,

            COUNT(DISTINCT cr.crash_record_id) as n_crash,

            SUM(cr.injuries_total) AS injuries,
            SUM(cr.injuries_fatal) AS injuries_fatal,


            wx.temp,
            wx.precip,
            wx.snow_1h,
            wx.rain_1h,
            
            v.violations,
            
            cg.congestion,
            
            cov.cases_total
            

    FROM ah
    LEFT JOIN wx
        ON wx.year = ah.year
        AND wx.month = ah.month
        AND wx.day = ah.day  
        AND wx.hour = ah.hour
    LEFT JOIN cr
        ON cr.year = ah.year
        AND cr.month = ah.month
        AND cr.day = ah.day
        AND cr.hour = ah.hour
    LEFT JOIN cg
        ON cg.year = ah.year
        AND cg.month = ah.month
        AND cg.day = ah.day
        AND cg.hour = ah.hour
    LEFT JOIN v
        ON v.year = ah.year
        AND v.month = ah.month
        AND v.day = ah.day
    LEFT JOIN cov
        ON cov.year = ah.year
        AND cov.month = ah.month
        AND cov.day = ah.day

    WHERE ah.datetime >= \'2017-09-01\' AND ah.datetime <= \'2021-01-31\'
    GROUP BY ah.year, ah.month, ah.day, ah.hour
    ORDER BY ah.year, ah.month, ah.day, ah.hour;
                                    ''', conn)

# Change date to speed up, but put it back to sept 1, 2017 when done

## Preprocess our data

In [ ]:
big_df.describe()

In [ ]:
big_df.info()

In [ ]:
all_df = big_df.copy()
all_df['cases_total'].fillna(0, inplace=True)
all_df['cases_total'] = all_df['cases_total'].astype(int)
all_df['violations'].fillna(0, inplace=True)
all_df['congestion'].fillna(0.1, inplace=True)
all_df['datetime'] = pd.to_datetime(all_df['datetime'])

all_df['month'] = all_df['month'].astype(str)
all_df['weekday'] = all_df['weekday'].astype(str)
all_df['hour'] = all_df['hour'].astype(str)


# all_df.columns
all_df.isna().sum()

In [ ]:
all_df.columns

In [ ]:
# treat data as bool (did we have a wreck that day at that intersection?)
# multiple crash days will be treated as one.  Wll
y = all_df['n_crash'].fillna(0).astype(int)

drop_cols = ['day', 'injuries', 
             'injuries_fatal', 'n_crash', 'week', 'datetime']

# categoricals: week, month, weekday

X = all_df.drop(columns=drop_cols) 


In [ ]:
X.columns

In [ ]:
all_df['n_crash'].value_counts()
#165550 / 5245
#5245 / 137
#137/10

X_ = pd.get_dummies(X)

drop_cols = ['snow_1h', 'hour_9', 'month_5', 'month_2', 'hour_23', 'month_8', 
             'month_7', 'month_1', 'hour_10', 'month_6', 'hour_21', 'month_10', 
             'hour_22', 'hour_20', 'month_4', 'hour_0']
X_ = X_.drop(columns=drop_cols) 


X_.info()


## Create our model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.4, random_state=0)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

tree_reg = DecisionTreeRegressor() 
tree_reg.fit(X_train, y_train)
tree_reg.score(X_test, y_test)  

My base tree model is worse than just taking the mean

In [ ]:
# SKIP THIS CELL UNLESS PERFORMING GRID Search



# # Create the parameter grid based on the results of random search 
# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [5, 8, 10],
#     'max_features': [12, 9, 6],
#     'min_samples_leaf': [3, 4, 5],
#     'min_samples_split': [4, 8, 12],
#     'n_estimators': [200, 400, 800]
# }
# # Create a based model
# rf = RandomForestRegressor()
# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=0)

# grid_search.fit(X_train, y_train)
# print(grid_search.best_estimator_)

In [ ]:
forest = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
                      max_features=15, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=10, min_samples_split=24,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=-1, oob_score=False, random_state=0,
                      verbose=0, warm_start=False)

forest.fit(X_train, y_train)
print(forest.score(X_test, y_test))  # quick feedback score for tuning

## Identify feature importance

In [ ]:
def plot_feature_importances(model, X_train):
    n_features = X_train.shape[1]
    
    importance = model.feature_importances_
    features = X_train.columns.values
    df = pd.DataFrame()
    df['imp'] = importance
    df['feat'] = features
    df = df.sort_values(by=['imp'], ascending=True)

    plt.figure(figsize=(8,len(df)//2))
    plt.barh(range(len(df)), df['imp'], align='center', color='purple') 
    plt.yticks(np.arange(len(df)), df['feat'])
    plt.xticks(rotation=45)
    plt.title('Feature Importance')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    return list(df['feat'])

In [ ]:
features = plot_feature_importances(forest, X_train)
print(features[:10])

## Plot our results and prediction

In [ ]:
# create an hourly plot of actual and predicted values.


days = ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
pred = forest.predict(X_)

plotme = all_df.copy()[['datetime', 'n_crash', 'weekday', 'precip', 'congestion', 'cases_total', 'violations', 'injuries', 'temp']]

plotme['pred'] = pred
plotme['MA'] = plotme.n_crash.rolling(5, min_periods=1, center=True, win_type='triang').mean()
plotme['error'] = abs(pred - plotme.n_crash)/plotme.n_crash

plotme.temp = plotme.temp.apply(lambda x: (x-273.15)*9/5+32) # K to F conversion temp
plotme.weekday = plotme.weekday.apply(lambda x: days[int(x)])



plt.figure(figsize=[30, 5])

plotme_range = plotme[plotme['datetime'] > datetime(2017, 9, 1)]


hover_list = ['Pred crash: %{customdata[8]:.2f}',
              'Actual crash: %{customdata[10]}',
              'Error: %{customdata[9]:.1%}',
                    'Date: %{customdata[0]}',
                    "Weekday: %{customdata[1]}",
                    "Injuries: %{customdata[2]}",
                    "Precip (mm): %{customdata[3]:.1f}",
                    "Temp (F): %{customdata[4]:.1f}",
                    "Congestion: %{customdata[5]:.2f}",
                    "Daily violations: %{customdata[6]}",
                    "Covid cases: %{customdata[7]}",   
             ]



fig = px.bar(plotme_range, 
             x='datetime', 
             y='n_crash', 
             title='Chicago Hourly Crash Model',
             
             )
  

fig.add_scatter(x=plotme_range['datetime'],
                y=plotme_range['MA'],
                mode='lines',
                hoverinfo='skip',
                name='moving avg.',
                line=dict(
                            color='rgba(220, 0, 0, 0.5)',
                            width=2
                        ),
                
    )
               
    
fig.add_scatter(x=plotme_range['datetime'], 
                y=plotme_range['pred'],
                mode='markers',
                name='model prediction',
                #marker_color='black',

                marker=dict(
                            color='rgb(0,255,0)',
                            size=5,
                            line=dict(width=1,
                                    color='DarkSlateGray'),
                            ),
                
                customdata=plotme_range[['datetime', 'weekday', 'injuries', 'precip', 'temp', 'congestion', 'violations', 'cases_total', 'pred', 'error', 'n_crash']],
                hovertemplate="<br>".join(hover_list),  # break between all items  

               )



fig.update_yaxes(range=[0, plotme_range['n_crash'].max()])
fig.update_layout(width=1200, height=500, legend=dict(x=0, y=1), hovermode='closest')
fig.show()

#fig.write_html('citywide_signalcrash_model.html')
fig.write_html('citywide_allcrash_model.html')

### Scoring metrics for the model

In [ ]:
print(y.describe())
print()
print('Cross validation score: {}'.format(cross_val_score(forest, X_, y, cv=5).mean())) # 2min
print('MAE: {:.2f}'.format(mean_absolute_error(forest.predict(X_), y)))
print('RMSE: {:.2f}'.format(np.sqrt(mean_squared_error(forest.predict(X_), y))))

## Results
With Random Forest Regressaion, we were able to:
- Achieve ~65% accuracy for daily crashes citywide (r2=60)
- Achieve >29% accuracy for daily signal crashes (smaller dataset) with MAE of ~1
- identify the most important variables in our model (congestion, violations, temp, year, covid_cases)

We were NOT able to improve the results using SMOTE.  Both ROC/AUC and score were worse, but wanted the code available if we decide to try the hourly data.